In [ ]:
# For modules loading
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from tqdm import tqdm
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, T5ForConditionalGeneration
import torch
from itertools import cycle
import os 
from transformers import pipeline

from rouge_score import rouge_scorer
import ast

# For BM25
import src.bm25_IR

### Loading devset

In [54]:
dev_set = pd.read_csv('output/devset/dev_set_w_IR.csv')

import ast
columns_to_convert = ['tf_idf', 'bm25', 'dense_cls', 'dense_max', 'dense_mean']

for col in columns_to_convert:
    dev_set[col] = dev_set[col].apply(ast.literal_eval)

In [64]:
dev_set.loc[0, 'tf_idf'][:3]

['Ejeren af en ejerlejlighed har sammen med andre ejere af lejligheder ejendomsret til grunden, fælles bestanddele og tilbehør m.v. efter et fordelingstal, der fastsættes som en brøkdel. Til lejligheden hører i samme forhold rettigheder og forpligtelser for ejeren som medlem af ejerforeningen. De i stk. 1 og 2 omhandlede rettigheder og forpligtelser kan ikke adskilles fra ejendomsretten til lejligheden.',
 'Hver ejerlejlighed anses som en selvstændig fast ejendom. Hver ejerlejlighed er identificeret ved eget identifikationsnummer.',
 'I denne lov forstås ved: Ejerlejlighed: En lejlighed samt andre særskilt afgrænsede husrum, der er registreret som en ejerlejlighed efter reglerne i denne lov eller tidligere gældende lovgivning. Ejerlejlighed: En lejlighed samt andre særskilt afgrænsede husrum, der er registreret som en ejerlejlighed efter reglerne i denne lov eller tidligere gældende lovgivning. Ejerforening: Et obligatorisk fællesskab bestående af samtlige ejere af ejerlejligheder på e

### KennethTM/gpt-neo-1.3B-danish

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model and tokenizer
MODEL_NAME = "KennethTM/gpt-neo-1.3B-danish"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# Set the device
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu" # set up for mac here, change to cuda if needed
model.to(DEVICE)

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj):

In [38]:
dev_set_subset = dev_set[20:25]

dev_set_subset

,question,answer,text,pnumber,law number,tf_idf,bm25,dense_cls,dense_max,dense_mean
20,Hvad skal sikringsmæssige foranstaltninger for...,"En sikringsplan, som skal indsendes til og god...",Sikringsmæssige foranstaltninger for nukleart ...,37 b,LBK nr 314 af 03/04/2017,['Beredskabsstyrelsen fører tilsyn med overhol...,['Sikringsmæssige foranstaltninger for nuklear...,"['Tilsyn med tilladelse efter § 2, stk. 1, 1. ...",['Ved gennemførelsen af anlægsprojektet nævnt ...,"['(Ophævet)', 'Distributøren skal sikre, at et..."
21,Hvornår kan Nævnet for Videnskabelig Uredeligh...,Hvis der foreligger en begrundet formodning om...,'Nævnet for Videnskabelig Uredelighed kan beha...,12,LOV nr 383 af 26/04/2017,['Nævnet for Videnskabelig Uredelighed kan beh...,['Nævnet for Videnskabelig Uredelighed kan beh...,['Nævnet for Videnskabelig Uredelighed kan som...,['Nævnet for Videnskabelig Uredelighed udarbej...,['Med bøde eller fængsel indtil 2 år straffes ...
22,"Hvornår kommer de i loven for livs-, ulykkes- ...","I alle tilfælde, hvor den begunstigede er inds...","'De i loven for livs-, ulykkes- og sygeforsikr...",NaN,LBK nr 1237 af 09/11/2015,"['De i loven for livs-, ulykkes- og sygeforsik...","['De i loven for livs-, ulykkes- og sygeforsik...","['De i loven for livs-, ulykkes- og sygeforsik...","['De i loven for livs-, ulykkes- og sygeforsik...","['Det udvalg, der er nævnt i § 17 i lov om men..."
23,Hvad sker der hvis to eller flere dataansvarli...,Så betragtes de som fælles dataansvarlige.,Fastsætter to eller flere dataansvarlige i fæl...,21,LOV nr 410 af 27/04/2017,['Fastsætter to eller flere dataansvarlige i f...,['Fastsætter to eller flere dataansvarlige i f...,['Fastsætter to eller flere dataansvarlige i f...,['Fastsætter to eller flere dataansvarlige i f...,['Friplejeboliger etableret ved nybyggeri elle...
24,"Bedriftværn, der er etableret i henhold til de...",Medmindre kommunalbestyrelsen efter ansøgning ...,Sikringsmæssige foranstaltninger for nukleart ...,81,LBK nr 314 af 03/04/2017,"['Bedriftværn, der er etableret i henhold til ...","['Bedriftværn, der er etableret i henhold til ...","['Bedriftværn, der er etableret i henhold til ...","['Bedriftværn, der er etableret i henhold til ...","['Når to tredjedele af anstaltstiden, dog mind..."


In [ ]:
neo_answers_tf_idf_k1 = []
# evaluating tf-idf
for question, documents in tqdm(zip(dev_set['question'], dev_set_subset['tf_idf_k1']), desc='Answering questions'):

    # assemble a prompt from the documents, question and prompting an answer
    prompt = f"Relevante paragraffer: {documents} Spørgsmål: {question} Indsæt svar her baseret på de relevante paragraffer:"

    # tokenize
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)

    max_length = len(input_ids[0]) + 100

    # generate an answer within torch.no_grad() to save compute
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            no_repeat_ngram_size=7,
            # generation set to stop at '.' as it otherwise just repeats itself (think it's because we don't sample)
            eos_token_id=tokenizer.encode(' Spørgsmål')[0]
        )

    # decode generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip(' Spørgsmål')

    # append answer to list
    neo_answers_tf_idf_k1.append(answer[len(prompt):].strip())  # strip the prompt to leave just the answer

Answering questions: 5it [01:35, 19.01s/it]


In [45]:
dev_set.loc[15, 'tf_idf']

"['Ethvert skib, der modtager isbryderhjælp fra en statskontrolleret isbryder, er undergivet dennes myndighed, så længe hjælpen ydes.', 'Ved statskontrolleret isbryder forstås i nærværende lov såvel det af staten chartrede isbrydningsmateriel som andre af istjenesten til isbrydning benyttede skibe. Ved chartring af isbrydningsmateriel, jf. stk. 1, er det en betingelse, at operatøren sikrer besætningen på de pågældende fartøjer løn og arbejdstid og andre arbejdsvilkår, som ikke er mindre gunstige, end hvad der fremgår af gældende danske kollektive overenskomster.', 'Denne lov træder i kraft den 15. april 1957. Samtidig ophæves lov nr. 251 af 13. juni 1922 om en isbryder og indførelse af isafgift samt lov nr. 131 af 18. april 1925 om tillæg til nævnte lov.']"

### strombergnlp/dant5-large

In [82]:
# Load the pretrained T5 model and tokenizer
model_name = "strombergnlp/dant5-large"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu" # set up for mac here, change to cuda if needed
model.to(DEVICE)


/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [ ]:
#for question, documents in tqdm(zip(dev_set['question, str'], dev_set['tf_idf_k1']), desc='Answering questions'):

t5_answers_tf_idf_k1 = []

# Example question and context

for question, documents in tqdm(zip(dev_set_subset['question'], dev_set_subset['tf_idf_k1']), desc='Answering questions'):
    
    # Format the input for T5
    input_text = f"Relevante paragraffer: {documents}\nSpørgsmål: {question}\nIndsæt svar her baseret på de relevante paragraffer:"

    # Tokenize the input and generate an answer
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(DEVICE)

    max_length = len(input_ids[0]) + 100

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            # generation set to stop at ' Spørgsmål' as it otherwise just repeats itself (think it's because we don't sample)
            eos_token_id=tokenizer.encode(' Spørgsmål')[0]
        )

    # Decode and print the generated answer
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip(' Spørgsmål')
    t5_answers_tf_idf_k1.append(answer)

Answering questions: 5it [00:32,  6.51s/it]


In [25]:
idx = 0

In [ ]:
print('Question:', dev_set_subset.loc[idx+5, 'question'])
print('Retrieved_doc:', dev_set.loc[idx+5, 'tf_idf_k1'])
print('Answer:', dev_set_subset.loc[idx+5, 'answer'])
print('\n')
print('T5:', t5_answers_tf_idf_k1[idx])
idx += 1

Question: Har en person, der er blevet videoafhørt pligt til at afgive forklaring som vidne under hovedforhandlingen?
Retrieved_doc: En person, der er blevet videoafhørt efter § 340 a eller § 155, stk. 3, har ikke pligt til at afgive forklaring som vidne under hovedforhandlingen. Retten kan helt undtagelsesvis pålægge en person, der er omfattet af stk. 1, at afgive forklaring som vidne under hovedforhandlingen, hvis dette er af afgørende betydning for sagens afgørelse og det ikke er tilstrækkeligt, at personen genafhøres til video.
Answer: En person, der er blevet videoafhørt efter, har ikke pligt til at afgive forklaring som vidne under hovedforhandlingen.




NameError: name 't5_answers_tf_idf_k1' is not defined